In [13]:
from selenium import webdriver
import re
import pandas as pd
import time
from tqdm import tqdm_notebook as tq

#driver = webdriver.Chrome()

In [14]:
def encode_count(out, base):
    return str(out) + str(int(base["1"])) + str(int(base["2"])) + str(int(base["3"]))

In [19]:
# dmenu
# ランナーは前、アウトカウントは後
def find_info(driver, name):
    res_list = []
    out = 0
    base = dict({"1": False, "2": False, "3": False})
    
    while True:
        runners = driver.find_element_by_xpath('//*[@id="liveArea"]/div/div[2]/div[3]/div[1]')
        base = dict({"1": False, "2": False, "3": False})
        for elem in runners.find_elements_by_class_name("on"):
            if "first" in elem.get_attribute("class"):
                base["1"] = True
            if "second" in elem.get_attribute("class"):
                base["2"] = True
            if "third" in elem.get_attribute("class"):
                base["3"] = True
                
        inning = driver.find_element_by_xpath('//*[@id="liveArea"]/div/div[2]/div[1]').text
                
        fielding_team = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[1]').text
        pitcher = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[4]/a').text
        pitcher_dominant_hand = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[5]').text
        era = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[6]/ul/li[1]').text[3:]
        num_pitching = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[6]/ul/li[2]').text[3:]
        num_strikeout = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[6]/ul/li[3]').text[3:]
        lost_points = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[1]/li[6]/ul/li[4]').text[2:]
        
        batting_team = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[1]').text
        batter = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[4]/a').text
        batter_dominant_bat = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[5]').text
        batting_average = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[6]/ul/li[1]').text[2:]
        num_homerun = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[6]/ul/li[2]').text[3:]
        points = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[6]/ul/li[3]').text[2:]
        num_fourball = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[6]/ul/li[4]').text[2:]
        num_steal = driver.find_element_by_xpath('//*[@id="liveArea"]/ul[2]/li[6]/ul/li[5]').text[2:]
        
        result = driver.find_element_by_xpath('//*[@id="liveArea"]/div/div[1]/table/tbody/tr/td[2]').text
        
        total_current_list = [encode_count(out, base), \
                              inning, \
                              fielding_team, pitcher, pitcher_dominant_hand, era, \
                              batting_team, batter, batter_dominant_bat, batting_average, num_homerun, points, num_fourball, num_steal, result]
        
        res_list.append(total_current_list)
        #print(encode_count(out, base))
        # アウトは更新後に更新
        out = len(driver.find_element_by_xpath('//*[@id="liveArea"]/div/div[2]/div[2]/table/tbody/tr[3]').find_elements_by_class_name("on"))
        if out == 3:
            res_list.append([encode_count(3, dict({"1": False, "2": False, "3": False}))])
            #print(encode_count(out, base))
            out = 0
                
        try:
            if driver.find_element_by_xpath('//*[@id="liveArea"]/table/tbody/tr/td[3]/p').text == "次打者":
                driver.find_element_by_xpath('//*[@id="liveArea"]/table/tbody/tr/td[3]/p').click() 
                #time.sleep(0.1)
            else:
                break
        except:
            #　ゲームセット
            break
        
    #return res_list
    columns = ["状況", "回", "守備チーム", "投手", "利き手", "防御率", "攻撃チーム", "打者", "打席", "打率", "本塁打数", "打点", "四球数", "盗塁数", "結果"]
    pd.DataFrame(res_list, dtype="object", columns=columns).to_csv("./game_num_data/"+name+".csv", encoding="cp932")

In [ ]:
#dmenu
today = "20180928"
for i in tq(range(200)):
    date = pd.to_datetime(today) - pd.Timedelta(days=i)
    date_string = str(date.year) + "{0:02d}".format(date.month) + "{0:02d}".format(date.day) + "0"
    for j in range(1, 7):
        new_date_string = date_string + str(j)
        #print(new_date_string)
        #if new_date_string in ["2018080701", "2018080702", "2018080703"]:
        #    continue
        url = "https://baseball.sports.smt.docomo.ne.jp/result/games/live_{}_01101.html".format(new_date_string)
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(1)
        try:
            find_info(driver, new_date_string)
        except:
            #raise
            pass
        driver.quit()

In [12]:
url = "https://baseball.sports.smt.docomo.ne.jp/result/games/live_{}_01101.html".format(new_date_string)
driver = webdriver.Chrome()
driver.get(url)

KeyboardInterrupt: 

In [ ]:
driver.find_element_by_xpath('//*[@id="liveArea"]/div/div[2]/div[1]').text

In [ ]:
a = [1, 2, 3]
b = [4, 5]
c = [a, b]

In [41]:
pd.DataFrame(c, columns=["a", "b", "c"])

,a,b,c
0,1,2,3.0
1,4,5,NaN
